## Deliverable 2. Create a Customer Travel Destinations Map.

In [48]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import maps_api_key

# Configure gmaps API key
gmaps.configure(api_key=maps_api_key)

In [49]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Ushuaia,AR,2022-02-13 20:53:16,-54.8000,-68.3000,40.66,81,100,39.12,moderate rain
1,1,Magadan,RU,2022-02-13 20:53:16,59.5667,150.8000,-5.19,67,40,9.64,scattered clouds
2,2,Kavieng,PG,2022-02-13 20:53:17,-2.5744,150.7967,81.09,78,97,6.31,overcast clouds
3,3,Kapaa,US,2022-02-13 20:53:17,22.0752,-159.3190,76.12,69,59,1.99,broken clouds
4,4,Port Elizabeth,ZA,2022-02-13 20:53:17,-33.9180,25.5701,70.09,78,0,11.50,clear sky


In [50]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [51]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
search_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
search_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
2,2,Kavieng,PG,2022-02-13 20:53:17,-2.5744,150.7967,81.09,78,97,6.31,overcast clouds
3,3,Kapaa,US,2022-02-13 20:53:17,22.0752,-159.3190,76.12,69,59,1.99,broken clouds
4,4,Port Elizabeth,ZA,2022-02-13 20:53:17,-33.9180,25.5701,70.09,78,0,11.50,clear sky
7,7,Rikitea,PF,2022-02-13 20:53:18,-23.1203,-134.9692,78.46,72,83,11.50,light rain
8,8,Hobart,AU,2022-02-13 20:52:19,-42.8794,147.3294,62.02,81,75,11.50,broken clouds
9,9,Puerto Ayora,EC,2022-02-13 20:53:19,-0.7393,-90.3518,80.56,88,91,5.99,overcast clouds
10,10,Hithadhoo,MV,2022-02-13 20:53:19,-0.6000,73.0833,81.81,75,77,9.44,broken clouds
15,15,Flinders,AU,2022-02-13 20:53:21,-34.5833,150.8552,70.38,79,100,4.00,overcast clouds
19,19,Avera,US,2022-02-13 20:53:23,33.1940,-82.5271,60.31,25,10,17.83,clear sky
23,23,Pathein,MM,2022-02-13 20:53:24,16.7833,94.7333,62.67,82,50,5.93,scattered clouds


In [52]:
len(search_cities_df)

327

In [53]:
# 4a. Determine if there are any empty rows.
search_cities_df.isnull().sum()


City_ID        0
City           0
Country        5
Date           0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [54]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
search_cities_df = search_cities_df.dropna()
search_cities_df.isnull().sum()

City_ID        0
City           0
Country        0
Date           0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [55]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = search_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Kavieng,PG,81.09,overcast clouds,-2.5744,150.7967,
3,Kapaa,US,76.12,broken clouds,22.0752,-159.3190,
4,Port Elizabeth,ZA,70.09,clear sky,-33.9180,25.5701,
7,Rikitea,PF,78.46,light rain,-23.1203,-134.9692,
8,Hobart,AU,62.02,broken clouds,-42.8794,147.3294,
9,Puerto Ayora,EC,80.56,overcast clouds,-0.7393,-90.3518,
10,Hithadhoo,MV,81.81,broken clouds,-0.6000,73.0833,
15,Flinders,AU,70.38,overcast clouds,-34.5833,150.8552,
19,Avera,US,60.31,clear sky,33.1940,-82.5271,
23,Pathein,MM,62.67,scattered clouds,16.7833,94.7333,


In [59]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": maps_api_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [65]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()


City           0
Country        0
Max Temp       0
Description    0
Lat            0
Lng            0
Hotel Name     0
dtype: int64

In [70]:
hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Kavieng,PG,81.09,overcast clouds,-2.5744,150.7967,Nusa Island Retreat
3,Kapaa,US,76.12,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
4,Port Elizabeth,ZA,70.09,clear sky,-33.9180,25.5701,39 On Nile Guest House
7,Rikitea,PF,78.46,light rain,-23.1203,-134.9692,People ThankYou
8,Hobart,AU,62.02,broken clouds,-42.8794,147.3294,Mantra on Collins Hobart
...,...,...,...,...,...,...,...
662,Auki,SB,77.09,light rain,-8.7676,160.7034,Auki Motel
664,Mount Gambier,AU,60.71,overcast clouds,-37.8333,140.7667,The Commodore
665,San Carlos De Bariloche,AR,85.75,clear sky,-41.1456,-71.3082,Hotel Patagonia
667,Kirakira,SB,79.84,light rain,-10.4544,161.9205,SanBiz Lodge


In [71]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [75]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [76]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))